# Loading packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import gc

from os import path
from matplotlib import pyplot as plt
from tensorflow.keras import layers, losses, optimizers, metrics, callbacks, Model, Input

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=9048)]
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

1 Physical GPU, 1 Logical GPUs


In [3]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 32

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Loading datasets

In [5]:
dataset = np.load("vgg_output.npy")
dataset.shape

(50000, 7, 7, 512)

In [6]:
filenames = list(os.walk(path.join(os.getcwd(), '..', 'cifar-10', 'train', 'train_images'), topdown=False))[0][2]
filenames = pd.DataFrame(filenames, columns=["name"]).reset_index(names="real_index")
filenames

,real_index,name
0,0,1.png
1,1,10.png
2,2,100.png
3,3,1000.png
4,4,10000.png
...,...,...
49995,49995,9995.png
49996,49996,9996.png
49997,49997,9997.png
49998,49998,9998.png


In [7]:
train_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'train.csv')).drop(["Unnamed: 0"], axis=1)
train_labels_df = train_labels_df.merge(filenames, on=["name"])
train_labels_df

,name,codes,real_index
0,1.png,6,0
1,100.png,1,2
2,10001.png,1,5
3,10002.png,6,6
4,10003.png,6,7
...,...,...,...
34995,9994.png,1,49994
34996,9996.png,3,49996
34997,9997.png,9,49997
34998,9998.png,1,49998


In [8]:
val_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'val.csv')).drop(["Unnamed: 0"], axis=1)
val_labels_df = val_labels_df.merge(filenames, on=["name"])
val_labels_df

,name,codes,real_index
0,10.png,3,1
1,1000.png,5,3
2,10000.png,5,4
3,10014.png,6,19
4,10029.png,7,35
...,...,...,...
4995,9961.png,8,49958
4996,9977.png,3,49975
4997,998.png,1,49978
4998,9986.png,0,49985


In [9]:
test_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'test.csv')).drop(["Unnamed: 0"], axis=1)
test_labels_df = test_labels_df.merge(filenames, on=["name"])
test_labels_df

,name,codes,real_index
0,10008.png,6,12
1,1001.png,9,14
2,10012.png,3,17
3,10015.png,5,20
4,10021.png,0,27
...,...,...,...
9995,9971.png,3,49969
9996,9981.png,6,49980
9997,9983.png,3,49982
9998,9984.png,0,49983


In [10]:
assert not train_labels_df.real_index.isin(test_labels_df).any()
assert not train_labels_df.real_index.isin(val_labels_df).any()
assert not test_labels_df.real_index.isin(val_labels_df).any()

In [11]:
train_x, train_y = dataset[train_labels_df.real_index, :], tf.one_hot(train_labels_df.codes, N_CLASS)
val_x, val_y = dataset[val_labels_df.real_index, :], tf.one_hot(val_labels_df.codes, N_CLASS)
test_x, test_y = dataset[test_labels_df.real_index, :], tf.one_hot(test_labels_df.codes, N_CLASS)
train_x.shape, val_x.shape, test_x.shape

((35000, 7, 7, 512), (5000, 7, 7, 512), (10000, 7, 7, 512))

In [12]:
del dataset
import gc
gc.collect()

0

# VGG

In [13]:
def create_model():
    input_layer = Input(shape=(7, 7, 512))
    x = layers.Flatten()(input_layer)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    output = layers.Dense(N_CLASS, activation="softmax")(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7, 7, 512)]       0         
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)                                             

In [14]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
)

checkpoint = callbacks.ModelCheckpoint(
    filepath="vgg_end.h5",
    monitor="val_categorical_accuracy",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, min_lr=0.00001, verbose=1)

In [15]:
history = model.fit(
    x=train_x, y=train_y,
    epochs=200,
    batch_size=256,
    validation_data=(val_x, val_y),
    shuffle=True,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/200
137/137 [==============================] - 4s 21ms/step - loss: 0.5060 - categorical_accuracy: 0.8362 - categorical_crossentropy: 0.5060 - val_loss: 0.4099 - val_categorical_accuracy: 0.8762 - val_categorical_crossentropy: 0.4099 - lr: 0.0010
Epoch 2/200
137/137 [==============================] - 2s 15ms/step - loss: 0.0918 - categorical_accuracy: 0.9704 - categorical_crossentropy: 0.0918 - val_loss: 0.4429 - val_categorical_accuracy: 0.8710 - val_categorical_crossentropy: 0.4429 - lr: 0.0010
Epoch 3/200
137/137 [==============================] - 2s 15ms/step - loss: 0.0281 - categorical_accuracy: 0.9919 - categorical_crossentropy: 0.0281 - val_loss: 0.5122 - val_categorical_accuracy: 0.8744 - val_categorical_crossentropy: 0.5122 - lr: 0.0010
Epoch 4/200
137/137 [==============================] - 2s 16ms/step - loss: 0.0209 - categorical_accuracy: 0.9928 - categorical_crossentropy: 0.0209 - val_loss: 0.5454 - val_categorical_accuracy: 0.8798 - val_categorical_crossentropy: 

In [16]:
gc.collect()

1470

# Experiments

Training will be repeated 10 times with different weights initialization.

In [17]:
TRAINING_SEEDS = list(range(10))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = create_model()

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss=losses.CategoricalCrossentropy(),
        metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
    )

    history = model.fit(
        x=train_x, y=train_y,
        epochs=200,
        batch_size=256,
        validation_data=(val_x, val_y),
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )
    eval_results = model.evaluate(x=test_x, batch_size=128, y=test_y)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]
    
    gc.collect()
    
results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv('vgg_results.csv')

Epoch 1/200
137/137 [==============================] - 4s 22ms/step - loss: 0.5103 - categorical_accuracy: 0.8379 - categorical_crossentropy: 0.5103 - val_loss: 0.4626 - val_categorical_accuracy: 0.8608 - val_categorical_crossentropy: 0.4626 - lr: 0.0010
Epoch 2/200
137/137 [==============================] - 2s 16ms/step - loss: 0.0957 - categorical_accuracy: 0.9693 - categorical_crossentropy: 0.0957 - val_loss: 0.4085 - val_categorical_accuracy: 0.8792 - val_categorical_crossentropy: 0.4085 - lr: 0.0010
Epoch 3/200
137/137 [==============================] - 2s 16ms/step - loss: 0.0274 - categorical_accuracy: 0.9917 - categorical_crossentropy: 0.0274 - val_loss: 0.4729 - val_categorical_accuracy: 0.8814 - val_categorical_crossentropy: 0.4729 - lr: 0.0010
Epoch 4/200
137/137 [==============================] - 2s 16ms/step - loss: 0.0158 - categorical_accuracy: 0.9952 - categorical_crossentropy: 0.0158 - val_loss: 0.5673 - val_categorical_accuracy: 0.8710 - val_categorical_crossentropy: 

In [18]:
results

,seed,loss,categorical_accuracy,categorical_crossentropy
0,0,0.628018,0.8850,0.628018
1,1,0.624239,0.8861,0.624239
2,2,0.609805,0.8907,0.609805
3,3,0.601188,0.8879,0.601188
4,4,0.629457,0.8851,0.629457
5,5,0.690771,0.8732,0.690771
6,6,0.608659,0.8907,0.608659
7,7,0.600445,0.8862,0.600445
8,8,0.615050,0.8875,0.615050
9,9,0.585364,0.8861,0.585364
